In [44]:
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import style
import numpy as np
import os
import mplfinance as mpf
import pandas as pd
import pandas_datareader.data as web
import bs4 as bs
import pickle
import requests
from collections import Counter
from sklearn import svm, neighbors
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [45]:
# X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)

In [46]:
def process_data_dor_labels(ticker):
    hm_days = 7
    df = pd.read_csv('sp500tickers_joined_closes.csv', index_col=0)
    df.columns = [x.split('.')[0] for x in df.columns]
    tickers=df.columns.values.tolist()
    df.fillna(0,inplace=True)
    
    for i in range(1,hm_days+1):
        df['{}_{}d'.format(ticker,i)]=(df[ticker].shift(-i) - df[ticker])/df[ticker]
        
    df.fillna(0,inplace=True)
    return tickers, df

# process_data_dor_labels('MO')

In [47]:
def buy_sell_hold(*args):
    cols = [c for c in args]
#     print(cols)
    requirement = 0.028
    for col in cols:
        if col > requirement:
            return 1
        if col < -requirement:
            return -1 
    return 0 

# buy_sell_hold(0.01, -0.03)

In [49]:
def extract_featuresets(ticker):
    hm_days = 7
    tickers, df = process_data_dor_labels(ticker)
    
    # if any of the price change > 0.02 in last 7 days (latest day), we buy 
    df['{}_target'.format(ticker)]=list(map(buy_sell_hold, 
                                            *[df['{}_{}d'.format(ticker, i)]for i in range(1, hm_days+1)]))
    
    vals = df['{}_target'.format(ticker)].values.tolist()
    str_vals = [str(i) for i in vals]
    print('Data spread:', Counter(str_vals))
    
    df.fillna(0,inplace=True)
    df=df.replace([np.inf,-np.inf],np.nan)
    df.dropna(inplace=True)
    
    df_vals = df[[ticker for ticker in tickers]].pct_change() #daiily price change for 500stocks
    df_vals=df_vals.replace([np.inf,-np.inf], 0)
    df_vals.fillna(0,inplace=True)
    
    X = df_vals.values # what the actual price change for all stocks
    y = df['{}_target'.format(ticker)].values # what our strategy tells us to do (buy, sell,hold)
    
    return X, y ,df

X, y ,df = extract_featuresets('MO')
tickers, dff = process_data_dor_labels('MO')
df['{}_target'.format('MO')]

Data spread: Counter({'0': 2132, '1': 1256, '-1': 889})


Date
2000-01-03    1
2000-01-04    1
2000-01-05    1
2000-01-06    1
2000-01-07    1
             ..
2016-12-23    0
2016-12-27    0
2016-12-28    0
2016-12-29    0
2016-12-30    0
Name: MO_target, Length: 4277, dtype: int64

In [38]:
def do_ml(ticker):
    X, y, df = extract_featuresets(ticker)
    
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)
    
    
#     clf = neighbors.KNeighborsClassifier()
#     clf = svm.LinearSVC()
    clf = RandomForestClassifier()   # set up the classifer  

    clf.fit(X_train, y_train)
    confidence = clf.score(X_test, y_test)
    prediction = clf.predict(X_test)
    
    print('Predicted spread:', Counter(prediction))
    print('Accuracy:', confidence)
    return confidence

do_ml('BAC')

Data spread: Counter({'1': 1485, '0': 1438, '-1': 1354})
Predicted spread: Counter({0: 431, 1: 355, -1: 284})
Accuracy: 0.4934579439252336


0.4934579439252336